In [1]:
import nltk
from nltk.tokenize import word_tokenize
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from collections import Counter
from datasets import load_dataset
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
import os

/Users/sohamchatterjee/miniforge3/envs/DL2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load some samples - Test split of SNLI

In [2]:
premise_text = "This church choir sings to the masses as they sing joyous songs from the book at a church."
hypothesis_text = "The church has cracks in the ceiling."
label = 'neutral'

# Baseline - Load model from pre-trained checkpoint and analyse on some samples

# Gold label - neutral

In [3]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
import torch.nn.functional as F
from nltk.tokenize import word_tokenize


word_to_idx = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/word_to_idx.pt")
pad_idx = word_to_idx.get('<pad>', 0)
unk_idx = word_to_idx.get('<unk>', 1)


def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    return tokens

def encode(tokens):
    return [word_to_idx.get(t, unk_idx) for t in tokens]


premise_tokens = preprocess_text(premise_text)
hypothesis_tokens = preprocess_text(hypothesis_text)
premise_encoded = torch.tensor(encode(premise_tokens))
hypothesis_encoded = torch.tensor(encode(hypothesis_tokens))

premise_padded = pad_sequence([premise_encoded], batch_first=True, padding_value=pad_idx)
hypothesis_padded = pad_sequence([hypothesis_encoded], batch_first=True, padding_value=pad_idx)


embedding_matrix = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/embedding_matrix.pt", weights_only=False)
embedding_layer = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float), freeze=True, padding_idx=pad_idx)

def combine(u, v):
    return torch.cat([u, v, torch.abs(u - v), u * v], dim=1)

class SentenceEncoder(nn.Module):
    def __init__(self, embedding_layer):
        super().__init__()
        self.embedding = embedding_layer

    def forward(self, x):
        embedded = self.embedding(x)
        mask = (x != pad_idx).unsqueeze(-1).float()
        summed = torch.sum(embedded * mask, dim=1)
        lengths = torch.sum(mask, dim=1)
        return summed / lengths.clamp(min=1)

class NLIClassifier(nn.Module):
    def __init__(self, encoder, input_dim, hidden_dim=512, output_dim=3):
        super().__init__()
        self.encoder = encoder
        self.fc1 = nn.Linear(4 * input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, premise, hypothesis):
        u = self.encoder(premise)
        v = self.encoder(hypothesis)
        combined = combine(u, v)
        x = F.relu(self.fc1(combined))
        return self.fc2(x)


hidden_dim = 300
model = NLIClassifier(SentenceEncoder(embedding_layer), input_dim=hidden_dim)
model.load_state_dict(torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/Baseline/snli_baseline_model.pth"))
model.eval()


with torch.no_grad():
    outputs = model(premise_padded, hypothesis_padded)
    predicted_class = torch.argmax(outputs, dim=1).item()


label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
print(f"Prediction: {label_map[predicted_class]}")


Prediction: entailment


# Baseline wrong

# Unidirectional LSTM

In [4]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
import torch.nn.functional as F
from nltk.tokenize import word_tokenize


word_to_idx = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/word_to_idx.pt")
pad_idx = word_to_idx.get('<pad>', 0)
unk_idx = word_to_idx.get('<unk>', 1)


def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    return tokens

def encode(tokens):
    return [word_to_idx.get(t, unk_idx) for t in tokens]


premise_tokens = preprocess_text(premise_text)
hypothesis_tokens = preprocess_text(hypothesis_text)
premise_encoded = torch.tensor(encode(premise_tokens))
hypothesis_encoded = torch.tensor(encode(hypothesis_tokens))


premise_padded = pad_sequence([premise_encoded], batch_first=True, padding_value=pad_idx)
hypothesis_padded = pad_sequence([hypothesis_encoded], batch_first=True, padding_value=pad_idx)


embedding_matrix = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/embedding_matrix.pt", weights_only=False)
embedding_layer = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float), freeze=True, padding_idx=pad_idx)

def combine(u, v):
    return torch.cat([u, v, torch.abs(u - v), u * v], dim=1)

class SentenceEncoder(nn.Module):
    def __init__(self, embedding_layer, hidden_dim=300):
        super().__init__()
        self.embedding = embedding_layer
        self.lstm = nn.LSTM(
            input_size=embedding_layer.embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=False
        )

    def forward(self, x):
        embedded = self.embedding(x)  
        lengths = (x != 0).sum(dim=1).clamp(min=1).cpu()  

        packed = nn.utils.rnn.pack_padded_sequence(
            embedded, lengths, batch_first=True, enforce_sorted=False
        )

        _, (h_n, _) = self.lstm(packed)  
        return h_n.squeeze(0) 

class NLIClassifier(nn.Module):
    def __init__(self, encoder, input_dim, hidden_dim=512, output_dim=3):
        super().__init__()
        self.encoder = encoder
        self.fc1 = nn.Linear(4 * input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, premise, hypothesis):
        u = self.encoder(premise)
        v = self.encoder(hypothesis)
        combined = combine(u, v)
        x = F.relu(self.fc1(combined))
        return self.fc2(x)


hidden_dim = 300
model = NLIClassifier(SentenceEncoder(embedding_layer), input_dim=hidden_dim)
model.load_state_dict(torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/v1/snli_v1_model.pth"))
model.eval()


with torch.no_grad():
    outputs = model(premise_padded, hypothesis_padded)
    predicted_class = torch.argmax(outputs, dim=1).item()


label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
print(f"Prediction: {label_map[predicted_class]}")


Prediction: contradiction


# Unidirectional LSTM wrong

# Bi LSTM

In [5]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
import torch.nn.functional as F
from nltk.tokenize import word_tokenize


word_to_idx = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/word_to_idx.pt")
pad_idx = word_to_idx.get('<pad>', 0)
unk_idx = word_to_idx.get('<unk>', 1)


def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    return tokens

def encode(tokens):
    return [word_to_idx.get(t, unk_idx) for t in tokens]


premise_tokens = preprocess_text(premise_text)
hypothesis_tokens = preprocess_text(hypothesis_text)
premise_encoded = torch.tensor(encode(premise_tokens))
hypothesis_encoded = torch.tensor(encode(hypothesis_tokens))


premise_padded = pad_sequence([premise_encoded], batch_first=True, padding_value=pad_idx)
hypothesis_padded = pad_sequence([hypothesis_encoded], batch_first=True, padding_value=pad_idx)


embedding_matrix = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/embedding_matrix.pt", weights_only=False)
embedding_layer = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float), freeze=True, padding_idx=pad_idx)

def combine(u, v):
    return torch.cat([u, v, torch.abs(u - v), u * v], dim=1)

class SentenceEncoder(nn.Module):
    def __init__(self, embedding_layer, hidden_dim=300):
        super().__init__()
        self.embedding = embedding_layer
        self.lstm = nn.LSTM(
            input_size=embedding_layer.embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=True  
        )

    def forward(self, x):
        embedded = self.embedding(x)  
        lengths = (x != 0).sum(dim=1).clamp(min=1).cpu()

        packed = nn.utils.rnn.pack_padded_sequence(
            embedded, lengths, batch_first=True, enforce_sorted=False
        )

        _, (h_n, _) = self.lstm(packed)  

        
        h_forward = h_n[0]  
        h_backward = h_n[1]  
        sentence_rep = torch.cat([h_forward, h_backward], dim=1)  
        return sentence_rep


class NLIClassifier(nn.Module):
    def __init__(self, encoder, input_dim, hidden_dim=512, output_dim=3):
        super().__init__()
        self.encoder = encoder
        self.fc1 = nn.Linear(4 * input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, premise, hypothesis):
        u = self.encoder(premise)
        v = self.encoder(hypothesis)
        combined = combine(u, v)
        x = F.relu(self.fc1(combined))
        return self.fc2(x)


hidden_dim = 300
model = NLIClassifier(SentenceEncoder(embedding_layer, hidden_dim), input_dim=2 * hidden_dim)
model.load_state_dict(torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/v2/snli_v2_model.pth"))
model.eval()


with torch.no_grad():
    outputs = model(premise_padded, hypothesis_padded)
    predicted_class = torch.argmax(outputs, dim=1).item()


label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
print(f"Prediction: {label_map[predicted_class]}")


Prediction: contradiction


# Bi LSTM wrong

# Bi-LSTM with max pooling 

In [6]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
import torch.nn.functional as F
from nltk.tokenize import word_tokenize


word_to_idx = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/word_to_idx.pt")
pad_idx = word_to_idx.get('<pad>', 0)
unk_idx = word_to_idx.get('<unk>', 1)


def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    return tokens

def encode(tokens):
    return [word_to_idx.get(t, unk_idx) for t in tokens]


premise_tokens = preprocess_text(premise_text)
hypothesis_tokens = preprocess_text(hypothesis_text)
premise_encoded = torch.tensor(encode(premise_tokens))
hypothesis_encoded = torch.tensor(encode(hypothesis_tokens))


premise_padded = pad_sequence([premise_encoded], batch_first=True, padding_value=pad_idx)
hypothesis_padded = pad_sequence([hypothesis_encoded], batch_first=True, padding_value=pad_idx)


embedding_matrix = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/embedding_matrix.pt", weights_only=False)
embedding_layer = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float), freeze=True, padding_idx=pad_idx)

def combine(u, v):
    return torch.cat([u, v, torch.abs(u - v), u * v], dim=1)

class SentenceEncoder(nn.Module):
    def __init__(self, embedding_layer, hidden_dim=300):
        super().__init__()
        self.embedding = embedding_layer
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(
            input_size=embedding_layer.embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=True
        )

    def forward(self, x):
        embedded = self.embedding(x)  
        lengths = (x != 0).sum(dim=1).clamp(min=1).cpu()

        
        packed = nn.utils.rnn.pack_padded_sequence(
            embedded, lengths, batch_first=True, enforce_sorted=False
        )

        
        packed_out, _ = self.lstm(packed)
        lstm_out, _ = nn.utils.rnn.pad_packed_sequence(packed_out, batch_first=True)
        sentence_rep, _ = torch.max(lstm_out, dim=1)  
        return sentence_rep

class NLIClassifier(nn.Module):
    def __init__(self, encoder, input_dim, hidden_dim=512, output_dim=3):
        super().__init__()
        self.encoder = encoder
        self.fc1 = nn.Linear(4 * input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, premise, hypothesis):
        u = self.encoder(premise)
        v = self.encoder(hypothesis)
        combined = combine(u, v)
        x = F.relu(self.fc1(combined))
        return self.fc2(x)


hidden_dim = 300
model = NLIClassifier(SentenceEncoder(embedding_layer, hidden_dim), input_dim=2 * hidden_dim)
model.load_state_dict(torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/v3/snli_v3_model.pth"))
model.eval()


with torch.no_grad():
    outputs = model(premise_padded, hypothesis_padded)
    predicted_class = torch.argmax(outputs, dim=1).item()


label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
print(f"Prediction: {label_map[predicted_class]}")


Prediction: contradiction


# BiLSTM with max pooling wrong

# All 4 models are wrong on this example. This might be because this is a hard sample

# Some specific examples

In [7]:
premise_text = "Two men sitting in the sun"
hypothesis_text = "Nobody is sitting in the shade"
label = 'neutral'

In [8]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
import torch.nn.functional as F
from nltk.tokenize import word_tokenize


word_to_idx = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/word_to_idx.pt")
pad_idx = word_to_idx.get('<pad>', 0)
unk_idx = word_to_idx.get('<unk>', 1)


def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    return tokens

def encode(tokens):
    return [word_to_idx.get(t, unk_idx) for t in tokens]


premise_tokens = preprocess_text(premise_text)
hypothesis_tokens = preprocess_text(hypothesis_text)
premise_encoded = torch.tensor(encode(premise_tokens))
hypothesis_encoded = torch.tensor(encode(hypothesis_tokens))

premise_padded = pad_sequence([premise_encoded], batch_first=True, padding_value=pad_idx)
hypothesis_padded = pad_sequence([hypothesis_encoded], batch_first=True, padding_value=pad_idx)


embedding_matrix = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/embedding_matrix.pt", weights_only=False)
embedding_layer = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float), freeze=True, padding_idx=pad_idx)

def combine(u, v):
    return torch.cat([u, v, torch.abs(u - v), u * v], dim=1)

class SentenceEncoder(nn.Module):
    def __init__(self, embedding_layer):
        super().__init__()
        self.embedding = embedding_layer

    def forward(self, x):
        embedded = self.embedding(x)
        mask = (x != pad_idx).unsqueeze(-1).float()
        summed = torch.sum(embedded * mask, dim=1)
        lengths = torch.sum(mask, dim=1)
        return summed / lengths.clamp(min=1)

class NLIClassifier(nn.Module):
    def __init__(self, encoder, input_dim, hidden_dim=512, output_dim=3):
        super().__init__()
        self.encoder = encoder
        self.fc1 = nn.Linear(4 * input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, premise, hypothesis):
        u = self.encoder(premise)
        v = self.encoder(hypothesis)
        combined = combine(u, v)
        x = F.relu(self.fc1(combined))
        return self.fc2(x)


hidden_dim = 300
model = NLIClassifier(SentenceEncoder(embedding_layer), input_dim=hidden_dim)
model.load_state_dict(torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/Baseline/snli_baseline_model.pth"))
model.eval()


with torch.no_grad():
    outputs = model(premise_padded, hypothesis_padded)
    predicted_class = torch.argmax(outputs, dim=1).item()


label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
print(f"Baseline Prediction: {label_map[predicted_class]}")



import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
import torch.nn.functional as F
from nltk.tokenize import word_tokenize


word_to_idx = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/word_to_idx.pt")
pad_idx = word_to_idx.get('<pad>', 0)
unk_idx = word_to_idx.get('<unk>', 1)


def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    return tokens

def encode(tokens):
    return [word_to_idx.get(t, unk_idx) for t in tokens]


premise_tokens = preprocess_text(premise_text)
hypothesis_tokens = preprocess_text(hypothesis_text)
premise_encoded = torch.tensor(encode(premise_tokens))
hypothesis_encoded = torch.tensor(encode(hypothesis_tokens))


premise_padded = pad_sequence([premise_encoded], batch_first=True, padding_value=pad_idx)
hypothesis_padded = pad_sequence([hypothesis_encoded], batch_first=True, padding_value=pad_idx)


embedding_matrix = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/embedding_matrix.pt", weights_only=False)
embedding_layer = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float), freeze=True, padding_idx=pad_idx)

def combine(u, v):
    return torch.cat([u, v, torch.abs(u - v), u * v], dim=1)

class SentenceEncoder(nn.Module):
    def __init__(self, embedding_layer, hidden_dim=300):
        super().__init__()
        self.embedding = embedding_layer
        self.lstm = nn.LSTM(
            input_size=embedding_layer.embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=False
        )

    def forward(self, x):
        embedded = self.embedding(x)  
        lengths = (x != 0).sum(dim=1).clamp(min=1).cpu()  

        packed = nn.utils.rnn.pack_padded_sequence(
            embedded, lengths, batch_first=True, enforce_sorted=False
        )

        _, (h_n, _) = self.lstm(packed)  
        return h_n.squeeze(0) 

class NLIClassifier(nn.Module):
    def __init__(self, encoder, input_dim, hidden_dim=512, output_dim=3):
        super().__init__()
        self.encoder = encoder
        self.fc1 = nn.Linear(4 * input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, premise, hypothesis):
        u = self.encoder(premise)
        v = self.encoder(hypothesis)
        combined = combine(u, v)
        x = F.relu(self.fc1(combined))
        return self.fc2(x)


hidden_dim = 300
model = NLIClassifier(SentenceEncoder(embedding_layer), input_dim=hidden_dim)
model.load_state_dict(torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/v1/snli_v1_model.pth"))
model.eval()


with torch.no_grad():
    outputs = model(premise_padded, hypothesis_padded)
    predicted_class = torch.argmax(outputs, dim=1).item()


label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
print(f"Uni LSTM Prediction: {label_map[predicted_class]}")

import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
import torch.nn.functional as F
from nltk.tokenize import word_tokenize


word_to_idx = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/word_to_idx.pt")
pad_idx = word_to_idx.get('<pad>', 0)
unk_idx = word_to_idx.get('<unk>', 1)


def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    return tokens

def encode(tokens):
    return [word_to_idx.get(t, unk_idx) for t in tokens]


premise_tokens = preprocess_text(premise_text)
hypothesis_tokens = preprocess_text(hypothesis_text)
premise_encoded = torch.tensor(encode(premise_tokens))
hypothesis_encoded = torch.tensor(encode(hypothesis_tokens))


premise_padded = pad_sequence([premise_encoded], batch_first=True, padding_value=pad_idx)
hypothesis_padded = pad_sequence([hypothesis_encoded], batch_first=True, padding_value=pad_idx)


embedding_matrix = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/embedding_matrix.pt", weights_only=False)
embedding_layer = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float), freeze=True, padding_idx=pad_idx)

def combine(u, v):
    return torch.cat([u, v, torch.abs(u - v), u * v], dim=1)

class SentenceEncoder(nn.Module):
    def __init__(self, embedding_layer, hidden_dim=300):
        super().__init__()
        self.embedding = embedding_layer
        self.lstm = nn.LSTM(
            input_size=embedding_layer.embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=True  
        )

    def forward(self, x):
        embedded = self.embedding(x)  
        lengths = (x != 0).sum(dim=1).clamp(min=1).cpu()

        packed = nn.utils.rnn.pack_padded_sequence(
            embedded, lengths, batch_first=True, enforce_sorted=False
        )

        _, (h_n, _) = self.lstm(packed)  

        
        h_forward = h_n[0]  
        h_backward = h_n[1]  
        sentence_rep = torch.cat([h_forward, h_backward], dim=1)  
        return sentence_rep


class NLIClassifier(nn.Module):
    def __init__(self, encoder, input_dim, hidden_dim=512, output_dim=3):
        super().__init__()
        self.encoder = encoder
        self.fc1 = nn.Linear(4 * input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, premise, hypothesis):
        u = self.encoder(premise)
        v = self.encoder(hypothesis)
        combined = combine(u, v)
        x = F.relu(self.fc1(combined))
        return self.fc2(x)


hidden_dim = 300
model = NLIClassifier(SentenceEncoder(embedding_layer, hidden_dim), input_dim=2 * hidden_dim)
model.load_state_dict(torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/v2/snli_v2_model.pth"))
model.eval()


with torch.no_grad():
    outputs = model(premise_padded, hypothesis_padded)
    predicted_class = torch.argmax(outputs, dim=1).item()


label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
print(f"BiLSTM Prediction: {label_map[predicted_class]}")


import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
import torch.nn.functional as F
from nltk.tokenize import word_tokenize


word_to_idx = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/word_to_idx.pt")
pad_idx = word_to_idx.get('<pad>', 0)
unk_idx = word_to_idx.get('<unk>', 1)


def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    return tokens

def encode(tokens):
    return [word_to_idx.get(t, unk_idx) for t in tokens]


premise_tokens = preprocess_text(premise_text)
hypothesis_tokens = preprocess_text(hypothesis_text)
premise_encoded = torch.tensor(encode(premise_tokens))
hypothesis_encoded = torch.tensor(encode(hypothesis_tokens))


premise_padded = pad_sequence([premise_encoded], batch_first=True, padding_value=pad_idx)
hypothesis_padded = pad_sequence([hypothesis_encoded], batch_first=True, padding_value=pad_idx)


embedding_matrix = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/embedding_matrix.pt", weights_only=False)
embedding_layer = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float), freeze=True, padding_idx=pad_idx)

def combine(u, v):
    return torch.cat([u, v, torch.abs(u - v), u * v], dim=1)

class SentenceEncoder(nn.Module):
    def __init__(self, embedding_layer, hidden_dim=300):
        super().__init__()
        self.embedding = embedding_layer
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(
            input_size=embedding_layer.embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=True
        )

    def forward(self, x):
        embedded = self.embedding(x)  
        lengths = (x != 0).sum(dim=1).clamp(min=1).cpu()

        
        packed = nn.utils.rnn.pack_padded_sequence(
            embedded, lengths, batch_first=True, enforce_sorted=False
        )

        
        packed_out, _ = self.lstm(packed)
        lstm_out, _ = nn.utils.rnn.pad_packed_sequence(packed_out, batch_first=True)
        sentence_rep, _ = torch.max(lstm_out, dim=1)  
        return sentence_rep

class NLIClassifier(nn.Module):
    def __init__(self, encoder, input_dim, hidden_dim=512, output_dim=3):
        super().__init__()
        self.encoder = encoder
        self.fc1 = nn.Linear(4 * input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, premise, hypothesis):
        u = self.encoder(premise)
        v = self.encoder(hypothesis)
        combined = combine(u, v)
        x = F.relu(self.fc1(combined))
        return self.fc2(x)


hidden_dim = 300
model = NLIClassifier(SentenceEncoder(embedding_layer, hidden_dim), input_dim=2 * hidden_dim)
model.load_state_dict(torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/v3/snli_v3_model.pth"))
model.eval()


with torch.no_grad():
    outputs = model(premise_padded, hypothesis_padded)
    predicted_class = torch.argmax(outputs, dim=1).item()


label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
print(f"Bi LSTM with max poolong Prediction: {label_map[predicted_class]}")


Baseline Prediction: entailment
Uni LSTM Prediction: contradiction
BiLSTM Prediction: contradiction
Bi LSTM with max poolong Prediction: contradiction


# As highlighted, all the LSTM based models predicted contradiction, whereas baseline predicts entailment

In [9]:
premise_text = "A man is walking a dog"
hypothesis_text = "No cat is outside"
label = 'neutral'

In [10]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
import torch.nn.functional as F
from nltk.tokenize import word_tokenize


word_to_idx = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/word_to_idx.pt")
pad_idx = word_to_idx.get('<pad>', 0)
unk_idx = word_to_idx.get('<unk>', 1)


def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    return tokens

def encode(tokens):
    return [word_to_idx.get(t, unk_idx) for t in tokens]


premise_tokens = preprocess_text(premise_text)
hypothesis_tokens = preprocess_text(hypothesis_text)
premise_encoded = torch.tensor(encode(premise_tokens))
hypothesis_encoded = torch.tensor(encode(hypothesis_tokens))

premise_padded = pad_sequence([premise_encoded], batch_first=True, padding_value=pad_idx)
hypothesis_padded = pad_sequence([hypothesis_encoded], batch_first=True, padding_value=pad_idx)


embedding_matrix = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/embedding_matrix.pt", weights_only=False)
embedding_layer = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float), freeze=True, padding_idx=pad_idx)

def combine(u, v):
    return torch.cat([u, v, torch.abs(u - v), u * v], dim=1)

class SentenceEncoder(nn.Module):
    def __init__(self, embedding_layer):
        super().__init__()
        self.embedding = embedding_layer

    def forward(self, x):
        embedded = self.embedding(x)
        mask = (x != pad_idx).unsqueeze(-1).float()
        summed = torch.sum(embedded * mask, dim=1)
        lengths = torch.sum(mask, dim=1)
        return summed / lengths.clamp(min=1)

class NLIClassifier(nn.Module):
    def __init__(self, encoder, input_dim, hidden_dim=512, output_dim=3):
        super().__init__()
        self.encoder = encoder
        self.fc1 = nn.Linear(4 * input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, premise, hypothesis):
        u = self.encoder(premise)
        v = self.encoder(hypothesis)
        combined = combine(u, v)
        x = F.relu(self.fc1(combined))
        return self.fc2(x)


hidden_dim = 300
model = NLIClassifier(SentenceEncoder(embedding_layer), input_dim=hidden_dim)
model.load_state_dict(torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/Baseline/snli_baseline_model.pth"))
model.eval()


with torch.no_grad():
    outputs = model(premise_padded, hypothesis_padded)
    predicted_class = torch.argmax(outputs, dim=1).item()


label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
print(f"Baseline Prediction: {label_map[predicted_class]}")



import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
import torch.nn.functional as F
from nltk.tokenize import word_tokenize


word_to_idx = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/word_to_idx.pt")
pad_idx = word_to_idx.get('<pad>', 0)
unk_idx = word_to_idx.get('<unk>', 1)


def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    return tokens

def encode(tokens):
    return [word_to_idx.get(t, unk_idx) for t in tokens]


premise_tokens = preprocess_text(premise_text)
hypothesis_tokens = preprocess_text(hypothesis_text)
premise_encoded = torch.tensor(encode(premise_tokens))
hypothesis_encoded = torch.tensor(encode(hypothesis_tokens))


premise_padded = pad_sequence([premise_encoded], batch_first=True, padding_value=pad_idx)
hypothesis_padded = pad_sequence([hypothesis_encoded], batch_first=True, padding_value=pad_idx)


embedding_matrix = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/embedding_matrix.pt", weights_only=False)
embedding_layer = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float), freeze=True, padding_idx=pad_idx)

def combine(u, v):
    return torch.cat([u, v, torch.abs(u - v), u * v], dim=1)

class SentenceEncoder(nn.Module):
    def __init__(self, embedding_layer, hidden_dim=300):
        super().__init__()
        self.embedding = embedding_layer
        self.lstm = nn.LSTM(
            input_size=embedding_layer.embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=False
        )

    def forward(self, x):
        embedded = self.embedding(x)  
        lengths = (x != 0).sum(dim=1).clamp(min=1).cpu()  

        packed = nn.utils.rnn.pack_padded_sequence(
            embedded, lengths, batch_first=True, enforce_sorted=False
        )

        _, (h_n, _) = self.lstm(packed)  
        return h_n.squeeze(0) 

class NLIClassifier(nn.Module):
    def __init__(self, encoder, input_dim, hidden_dim=512, output_dim=3):
        super().__init__()
        self.encoder = encoder
        self.fc1 = nn.Linear(4 * input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, premise, hypothesis):
        u = self.encoder(premise)
        v = self.encoder(hypothesis)
        combined = combine(u, v)
        x = F.relu(self.fc1(combined))
        return self.fc2(x)


hidden_dim = 300
model = NLIClassifier(SentenceEncoder(embedding_layer), input_dim=hidden_dim)
model.load_state_dict(torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/v1/snli_v1_model.pth"))
model.eval()


with torch.no_grad():
    outputs = model(premise_padded, hypothesis_padded)
    predicted_class = torch.argmax(outputs, dim=1).item()


label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
print(f"Uni LSTM Prediction: {label_map[predicted_class]}")

import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
import torch.nn.functional as F
from nltk.tokenize import word_tokenize


word_to_idx = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/word_to_idx.pt")
pad_idx = word_to_idx.get('<pad>', 0)
unk_idx = word_to_idx.get('<unk>', 1)


def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    return tokens

def encode(tokens):
    return [word_to_idx.get(t, unk_idx) for t in tokens]


premise_tokens = preprocess_text(premise_text)
hypothesis_tokens = preprocess_text(hypothesis_text)
premise_encoded = torch.tensor(encode(premise_tokens))
hypothesis_encoded = torch.tensor(encode(hypothesis_tokens))


premise_padded = pad_sequence([premise_encoded], batch_first=True, padding_value=pad_idx)
hypothesis_padded = pad_sequence([hypothesis_encoded], batch_first=True, padding_value=pad_idx)


embedding_matrix = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/embedding_matrix.pt", weights_only=False)
embedding_layer = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float), freeze=True, padding_idx=pad_idx)

def combine(u, v):
    return torch.cat([u, v, torch.abs(u - v), u * v], dim=1)

class SentenceEncoder(nn.Module):
    def __init__(self, embedding_layer, hidden_dim=300):
        super().__init__()
        self.embedding = embedding_layer
        self.lstm = nn.LSTM(
            input_size=embedding_layer.embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=True  
        )

    def forward(self, x):
        embedded = self.embedding(x)  
        lengths = (x != 0).sum(dim=1).clamp(min=1).cpu()

        packed = nn.utils.rnn.pack_padded_sequence(
            embedded, lengths, batch_first=True, enforce_sorted=False
        )

        _, (h_n, _) = self.lstm(packed)  

        
        h_forward = h_n[0]  
        h_backward = h_n[1]  
        sentence_rep = torch.cat([h_forward, h_backward], dim=1)  
        return sentence_rep


class NLIClassifier(nn.Module):
    def __init__(self, encoder, input_dim, hidden_dim=512, output_dim=3):
        super().__init__()
        self.encoder = encoder
        self.fc1 = nn.Linear(4 * input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, premise, hypothesis):
        u = self.encoder(premise)
        v = self.encoder(hypothesis)
        combined = combine(u, v)
        x = F.relu(self.fc1(combined))
        return self.fc2(x)


hidden_dim = 300
model = NLIClassifier(SentenceEncoder(embedding_layer, hidden_dim), input_dim=2 * hidden_dim)
model.load_state_dict(torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/v2/snli_v2_model.pth"))
model.eval()


with torch.no_grad():
    outputs = model(premise_padded, hypothesis_padded)
    predicted_class = torch.argmax(outputs, dim=1).item()


label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
print(f"BiLSTM Prediction: {label_map[predicted_class]}")


import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
import torch.nn.functional as F
from nltk.tokenize import word_tokenize


word_to_idx = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/word_to_idx.pt")
pad_idx = word_to_idx.get('<pad>', 0)
unk_idx = word_to_idx.get('<unk>', 1)


def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    return tokens

def encode(tokens):
    return [word_to_idx.get(t, unk_idx) for t in tokens]


premise_tokens = preprocess_text(premise_text)
hypothesis_tokens = preprocess_text(hypothesis_text)
premise_encoded = torch.tensor(encode(premise_tokens))
hypothesis_encoded = torch.tensor(encode(hypothesis_tokens))


premise_padded = pad_sequence([premise_encoded], batch_first=True, padding_value=pad_idx)
hypothesis_padded = pad_sequence([hypothesis_encoded], batch_first=True, padding_value=pad_idx)


embedding_matrix = torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/embedding_matrix.pt", weights_only=False)
embedding_layer = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float), freeze=True, padding_idx=pad_idx)

def combine(u, v):
    return torch.cat([u, v, torch.abs(u - v), u * v], dim=1)

class SentenceEncoder(nn.Module):
    def __init__(self, embedding_layer, hidden_dim=300):
        super().__init__()
        self.embedding = embedding_layer
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(
            input_size=embedding_layer.embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=True
        )

    def forward(self, x):
        embedded = self.embedding(x)  
        lengths = (x != 0).sum(dim=1).clamp(min=1).cpu()

        
        packed = nn.utils.rnn.pack_padded_sequence(
            embedded, lengths, batch_first=True, enforce_sorted=False
        )

        
        packed_out, _ = self.lstm(packed)
        lstm_out, _ = nn.utils.rnn.pad_packed_sequence(packed_out, batch_first=True)
        sentence_rep, _ = torch.max(lstm_out, dim=1)  
        return sentence_rep

class NLIClassifier(nn.Module):
    def __init__(self, encoder, input_dim, hidden_dim=512, output_dim=3):
        super().__init__()
        self.encoder = encoder
        self.fc1 = nn.Linear(4 * input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, premise, hypothesis):
        u = self.encoder(premise)
        v = self.encoder(hypothesis)
        combined = combine(u, v)
        x = F.relu(self.fc1(combined))
        return self.fc2(x)


hidden_dim = 300
model = NLIClassifier(SentenceEncoder(embedding_layer, hidden_dim), input_dim=2 * hidden_dim)
model.load_state_dict(torch.load("/Users/sohamchatterjee/Documents/UvA/ATCS/Practical_1/Checkpoints/chatty_atcs/v3/snli_v3_model.pth"))
model.eval()


with torch.no_grad():
    outputs = model(premise_padded, hypothesis_padded)
    predicted_class = torch.argmax(outputs, dim=1).item()


label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}
print(f"Bi LSTM with max poolong Prediction: {label_map[predicted_class]}")


Baseline Prediction: entailment
Uni LSTM Prediction: contradiction
BiLSTM Prediction: contradiction
Bi LSTM with max poolong Prediction: contradiction


# As highlighted, all the LSTM based models predicted contradiction, whereas baseline predicts entailment

 # In the first example, the models predicted Contradiction likely due to the contradictory nature of the phrase "Two men sitting" and "Nobody is sitting". The models missed the representation for sun and shade

 # In the second example, the models predicted Contradiction likely due to the contradictory nature of the phrase "A man is walking" and "No cat is outside". The models thought that walking is a thing usually done outside, but the hypothesis highlights that no cat is outside. Model failed to identify the difference between a dog being walked outside vs no cat present outside. Model assumes a negative correlation/contradiction, whereas in reality, the premise and hypothesis are completely independent of each other and hence neutral. 